In [94]:
import pandas as pd
import numpy as np
import collections
import math
import pymorphy2

In [96]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [135]:
def calculation_tfidf(array):
    
    def description_processing(line):
        
        a = ['.', '-', '(', ')', ',', ':', ';']
        
        for i in range(0, len(a)):
        
            line = line.replace(a[i], ' ')
            
        line = line.split()
    
        return line

    def calculation_tf(text):
        
        tf_text = collections.Counter(text)
    
        for i in tf_text:
        
            tf_text[i] = tf_text[i]/float(len(text))
        
        return tf_text

    def calculation_idf(word, array):
        
        n = 0
        
        for i in array:
            
            if word in i:
                
                n += 1
                
        return math.log10(len(array)/n)

    documents_list = []

    for line in array:

        tf_idf_dictionary = {}

        calculated_tf = calculation_tf(description_processing(line))

        for word in calculated_tf:
            
            if (calculated_tf[word] * calculation_idf(word, array)) > 0.03:
                
                tf_idf_dictionary[word] = calculated_tf[word] * calculation_idf(word, array)
            
        documents_list.append(tf_idf_dictionary)

    return documents_list

In [137]:
data_train = pd.read_csv('train.csv')
data = data_train['name'] + ' ' + data_train['description']

In [152]:
big = calculation_tfidf(data)

best = {'Менеджер': {},
        'Искусство': {},
        'Рабочий': {},
        'Дизайнер': {},
        'Специалист': {},
        'СМИ': {},
        'Врач': {},
        'other': {},
        'Инженер': {},
        'IT': {},
        'Право': {},
        'Учитель': {},
        'Агент': {}}

for i in range(0, len(big)):
    for key in big[i].keys():
        key = lemmatize(key)[0]
        if key not in best.get(data_train['category'][i]):
            (best.get(data_train['category'][i])).update({key: 1})
        else:
            (best.get(data_train['category'][i])).update({key: (best.get(data_train['category'][i])).get(key) + 1})
            
best_list = []

for big_key in best.keys():
    a = []
    for small_key in best[big_key].keys():
        if (best[big_key]).get(small_key) >= 4:
            a.append(small_key)
            
    if len(a) <= 20:
        for small_key in best[big_key].keys():
            if len(a) <= 20:
                if (best[big_key]).get(small_key) >= 2:
                    a.append(small_key)
        

                         
    best_list.append(a)

In [155]:
data_test = pd.read_csv('test.csv')
data_test =  data_test['name'] + ' ' + data_test['description']

In [156]:
pred = []
for i in range(0, len(data_test)):
    a = 0
    k = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for word in description_processing(data_test[i]):
        
        word = (morph.parse(word)[0]).normal_form
            
        for j in range(0, 13):
            if word in best_list[j]:
                k[j] = k[j] + 1
    for j in range(0, 13):
        if k[j] == np.max(k) and a == 0:
            pred.append(pd.unique(data_train['category'])[j])
            a = 1

In [157]:
Id = []
pred = np.array(pred)
for i in range (0, 106):
    Id.append(i)
Id = np.reshape(Id, pred.shape)
Answer = pd.DataFrame(Id)
Answer.columns = ['id']
Answer['category'] = pred
Answer.to_csv("Answer.csv", index=False)